In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data=pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
train_data.head()

In [ ]:
test_data=pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
test_data.head()

In [ ]:
# train datani boluruk x ve y
X=train_data.iloc[:, :-1]
y=train_data.iloc[:, -1]

In [ ]:
# test ve trainin Xinden esas data yaradiriq
data=pd.concat([X, test_data], ignore_index=True)
data.head()

In [ ]:
# esas data id, cateqoriya ve saylari secirik
identifier=data.select_dtypes(include=['int64'])
categorical = data.select_dtypes(include=[object])
numerical=data.select_dtypes(include=['float64'])

In [ ]:
# cateqoriyalari 5 faizden az olani other edirik
for column in categorical:
    percents=categorical[column].value_counts(normalize=True)*100
    other_classes=list(percents[percents<5.0].index)
    categorical[column]=categorical[column].replace(other_classes,'other')

In [ ]:
# ve butun cateqoriyalari one hot edirik
categorical=pd.get_dummies(categorical)
categorical.shape

In [ ]:
categorical

In [ ]:
# 95 faiz eyni classli olan feautrelari silmek 
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=(.95 * (1 - .95)))
categorical=selector.fit_transform(categorical)

In [ ]:
categorical.shape

In [ ]:
# categricali dataframe'a cevirmek
categorical=pd.DataFrame(categorical)
categorical.head()

In [ ]:
categorical

In [ ]:
# sonra kateqoriya ve id'ni birlesdiririk
cat_and_id_data=pd.concat([identifier, categorical], axis=1)
cat_and_id_data.head(20)

In [ ]:
#sonra bu birlesmi datani train hissesini kesirik
cat_and_id_data_train=cat_and_id_data.iloc[:300000,:]

In [ ]:
#sonra bu birlesmi datani test hissesini kesirik
cat_and_id_data_test=cat_and_id_data.iloc[300000:, :]

In [ ]:
# ve numericali da train ve test hisselere boluruk
numerical_train=numerical.iloc[:300000,:]
numerical_test=numerical.iloc[300000:, :]

In [ ]:
# ve train ve validasiya hisseleri yaratmaq ucun train datani birlesdririk
train_data=pd.concat([cat_and_id_data_train, numerical_train], axis=1)
train_data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# train datani train ve validasiyaya boluruk
X_train, X_valid, y_train, y_valid = train_test_split(train_data, y, test_size=0.33, random_state=42)

In [ ]:
# ve alinan 2 datadan catgorical datalari gotururuk
X_train_cat=X_train.select_dtypes(include=['uint8'])
X_test_cat=X_valid.select_dtypes(include=['uint8'])

In [ ]:
# ve alinan 2 datadan numerical datalari gotururuk
X_train_num=X_train.select_dtypes(include=['float64'])
X_test_num=X_valid.select_dtypes(include=['float64'])

In [ ]:
# train ve valid datalar scaling olunur
minmaxscaler=MinMaxScaler()
minmaxscaler.fit(X_train_num)
X_train_num=minmaxscaler.transform(X_train_num)
X_test_num=minmaxscaler.transform(X_test_num)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import QuantileTransformer
quantile = QuantileTransformer(output_distribution='normal')

In [ ]:
#train data quantiling olunur
X_train_num = quantile.fit_transform(X_train_num)

In [ ]:
#valid data da quantiling olunur
X_test_num = quantile.fit_transform(X_test_num)


In [ ]:
# ve her ikisinden dataframe yaradilir ki bilinsin ki her sey qaydasindadir mi 
# (bunu plotting ederek de yoxlamaq olar)
df_X_train_num=pd.DataFrame(X_train_num)
df_X_test_num=pd.DataFrame(X_test_num)

In [ ]:
# cateqocial train datanin indexleri
df_X_train_cat_indices=pd.DataFrame(X_train_cat.index)
df_X_train_cat_indices

In [ ]:
# numerical train datada indexler columnu yaradilir
df_X_train_num['cat_indices']=df_X_train_cat_indices
df_X_train_num

In [ ]:
# train datanin numericalina duzgun indexleri yaziriq ki cateqoricalnan birlesdirmek olsun
df_X_train_num=df_X_train_num.set_index('cat_indices')
df_X_train_num

In [ ]:
# ve ikisini birlesdirib bir train data aliriq
df_X_train=pd.concat([X_train_cat, df_X_train_num], axis=1)
df_X_train

In [ ]:
# cateqocial test datanin indexleri
df_X_test_cat_indices=pd.DataFrame(X_test_cat.index)
df_X_test_cat_indices

In [ ]:
# numerical test datada indexler columnu yaradilir
df_X_test_num['cat_indices']=df_X_test_cat_indices
df_X_test_num

In [ ]:
# test datanin numericalina duzgun indexleri yaziriq ki cateqoricalnan birlesdirmek olsun
df_X_test_num=df_X_test_num.set_index('cat_indices')
df_X_test_num

In [ ]:
# ve ikisini birlesdirib bir test data aliriq
df_X_test=pd.concat([X_test_cat, df_X_test_num], axis=1)
df_X_test

In [ ]:
# train data 
# df_X_train, y_train
# test data
# df_X_test, y_valid
#train categorical
# X_train_cat
#train numerical
# df_X_train_num
#test categorical
# X_test_cat
#test numercial
# df_X_test_num


In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

In [ ]:
# f_regression hamisini eyni gosterdi, mutual infoda ki islemedi
new=SelectKBest(f_regression, k='all')
new.fit(df_X_train_num,y_train)
new.scores_

In [ ]:
# esas code budur
for i in range(len(new.scores_)):
    print('Feature %d: %f' % (i, new.scores_[i]))


In [ ]:
# barplot ise hansilari onemli olduqlarini vizual gosterir
plt.bar([i for i in range(len(new.scores_))], new.scores_)
plt.show()

In [ ]:
# bu kod her bir elementin en boyuyune nisbeten faiz nisbetini gosterir
for i in range (0, len(new.scores_)):
    print(new.scores_[i]/new.scores_.max()*100)

In [ ]:
# RFEni import etmek
from sklearn.feature_selection import RFE

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
# rfeni linear regressionda yoxlamaq
model = LinearRegression()
rfe = RFE(model, n_features_to_select=9)
rfe = rfe.fit(df_X_train_num,y_train)

In [ ]:
# rankinglere baxmaq (f_regressionun secdiyi columnlarla duz gelir)
rfe.ranking_

In [ ]:
# asagidaki iterative koddan 0.05den kicil=kleri silsek 13 dene column qalir, ona gore de 13 yaziriq
cat_model = LinearRegression()
cat_rfe = RFE(cat_model, n_features_to_select=15)
cat_rfe = cat_rfe.fit(X_train_cat,y_train)

In [ ]:
cat_rfe.ranking_

In [ ]:
# saxlanilmali columnlar
z=[1,2,3,4,5,1,1,1,1]
for i in range(0, len(z)):
    if z[i] != 1:
        print(i, end = " ")

In [ ]:
for i in range(0, len(rfe.ranking_)):
    if rfe.ranking_[i] != 1:
        print(i, end = " ")

In [ ]:
train_data=train_data.drop(columns=[cont2, 3, 4, 9, 13 ])
train_data

In [ ]:
df_X_test_num=df_X_test_num.drop(columns=[2, 3, 4, 9, 13 ])
df_X_test_num

In [ ]:
for i in range(0, len(cat_rfe.ranking_)):
    if cat_rfe.ranking_[i] != 1:
        print(i, end = " ")

In [ ]:
X_train_cat=X_train_cat.drop(columns=[2, 3, 4, 9, 13 ])
X_train_cat

In [ ]:
X_test_cat=X_test_cat.drop(columns=[2, 3, 4, 9, 13 ])
X_test_cat

In [ ]:
df_X_train=pd.concat([X_train_cat, df_X_train_num], axis=1)
df_X_train

In [ ]:
train_data=pd.concat([cat_and_id_data_train, numerical_train], axis=1)
train_data

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
###

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.pipeline import make_pipeline
#clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))
cross_val_score(clf, X, y, cv=cv)
#pipe = make_pipeline(QuantileTransformer(), LogisticRegression())